In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

M = 4
# Create 4 single-server queues with FCFS scheduling
node = np.empty(4, dtype=object)
node[0] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[1] = Queue(model, 'Queue2', SchedStrategy.FCFS)
node[2] = Queue(model, 'Queue3', SchedStrategy.FCFS)
node[3] = Queue(model, 'Queue4', SchedStrategy.FCFS)  # only closed classes

source = Source(model, 'Source')
sink = Sink(model, 'Sink')

# Create job classes
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 100, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

# Set service rates
for i in range(M):
    node[i].set_service(jobclass[0], Exp(i+1))
    node[i].set_service(jobclass[1], Exp(np.sqrt(i+1)))

# Set arrival rate for open class using APH distribution
source.set_arrival(jobclass[1], APH.fit_mean_and_scv(3, 64))

In [3]:
# Setup routing matrix
M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()

# Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)
P.set(jobclass[0], jobclass[0], node[2], node[3], 1.0)
P.set(jobclass[0], jobclass[0], node[3], node[0], 1.0)

# Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Sink (excludes Queue4)
P.set(jobclass[1], jobclass[1], source, node[0], 1.0)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)
P.set(jobclass[1], jobclass[1], node[2], sink, 1.0)

model.link(P)

In [ ]:
print('This example shows the execution of the solver on a 2-class mixed model with 4 single server nodes.')

# Create solvers
solver = np.array([], dtype=object)
# Note: CTMC is infinite on this model due to high population so omitted CTMC and SSA
solver = np.append(solver, JMT(model, cutoff=3, keep=False, verbose=True, seed=23000, samples=20000))
#solver = np.append(solver, FLD(model, keep=False, verbose=True, seed=23000)) #TODO Fluid is very slow
solver = np.append(solver, MVA(model, method='lin', keep=False, verbose=True, seed=23000))
solver = np.append(solver, NC(model, keep=False, verbose=True, seed=23000))
# Note: MAM does not support mixed models with both open and closed classes
solver = np.append(solver, DES(model, cutoff=3, keep=False, verbose=True, seed=23000, samples=20000))

# Solve and display results
AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].avg_table()